<a href="https://colab.research.google.com/github/Mightysam01/predict_health_costs_with_regression/blob/main/Copy_of_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
import os

from urllib.request import Request, urlopen, urlretrieve
from sklearn.utils import shuffle

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.head()

In [ ]:
dataset.isna().sum()

this shows we don't have NaN values in the dataset

In [ ]:
dataset.describe().transpose()

Classifying the unique values in each column if greater or less than 10

In [ ]:
for column in dataset:
  if dataset[column].nunique() < 10:
    print(column, '=', pd.unique(dataset[column]))

  else:
    print(column, 'have', dataset[column].nunique(), 'different values.')

checking each column datatype so we can differentiate between the categorical and numerical data

In [ ]:
dataset.dtypes

- four numerical columns (age, bmi, children and expenses)
- three categorical columns (sex, smoker and region)
- the categorical data needs to be converted to numerical data so they can fit into the model

In [ ]:
dataset['sex'].replace(
    ['female', 'male'],
    [0, 1],
    inplace=True
)
dataset['smoker'].replace(
    ['yes', 'no'],
    [0, 1],
    inplace=True
)
dataset['region'].replace(
    ['southwest', 'southeast', 'northwest', 'northeast'],
    [0, 1, 2, 3],
    inplace=True
)

shuffle the dataset

In [ ]:
dataset = shuffle(dataset).reset_index(drop=True)

In [ ]:
dataset.tail()

Now that the categorical data has been converted to numerical data;
- 80% of the dataset will be used as the train_dataset
- 20% of the dataset will be used as the test_dataset

In [ ]:
dataset.shape[0]

In [ ]:
train_dataset = dataset[0:int(0.8*dataset.shape[0])]
train_dataset.shape

In [ ]:
train_dataset.tail()

In [ ]:
test_dataset = dataset[int(0.8*dataset.shape[0]):int(dataset.shape[0])]
test_dataset.shape

In [ ]:
test_dataset.head()

Defining the train and test labels by 'pop' the 'expenses'

In [ ]:
train_labels = train_dataset.pop('expenses')

In [ ]:
test_labels = test_dataset.pop('expenses')

Creating the Model

Normalizing the training dataset

In [ ]:
normalizer = layers.experimental.preprocessing.Normalization()

In [ ]:
normalizer.adapt(np.array(train_dataset))

Building the model

In [ ]:
model = keras.Sequential([
    normalizer,
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])

Compiling the Model

In [ ]:
model.compile(
    optimizer = tf.optimizers.Adam(learning_rate=0.1),
    loss = 'mae',
    metrics = ['mae', 'mse']
)

In [ ]:
model.build()

In [ ]:
model.summary()

In [ ]:
History = model.fit(
    train_dataset,
    train_labels,
    epochs = 100
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
